# 利用 BeautifulSoup 拆解法 - 蘋果日報關鍵字查詢為例 - 詳文

In [26]:
#coding:utf-8
#65001
import urllib.request
import json
import codecs
import sys
import argparse as ap
import time
import datetime
import requests
import random
from bs4 import BeautifulSoup as bs
from urllib.parse import quote

#python main.py 八仙塵爆 2015-06-27 2015-08-24 1
#def argParse():
#    parser=ap.ArgumentParser(description='Apple Daily News Crawler')
#    parser.add_argument("keyword", help="Serch Keyword")
#    parser.add_argument("start_date", help="Start (2017-01-01)")
#    parser.add_argument("end_date", help="End (2017-02-02)" )
#    parser.add_argument("pages", help="Pages")
#    return parser.parse_args()

#args=argParse()
#keyword = args.keyword
#start_date = args.start_date
#end_date = args.end_date
#pages = args.pages

keyword = '肺炎'
start_date = '2020-04-14'
end_date = '2020-04-15'
pages = '1'

rs = requests.session()

def start_requests(uri):
    if( len(start_date.split("-"))==3 and len(end_date.split("-"))==3) :
        for i in range(1,int(pages)+1):
            str_page = ''+('%s' % i)
            str_rand = str(random.randint(1,99999))
            print (uri+", page="+str_page)
            parseAppleDailyNews(uri+"&page="+str_page)
            time.sleep(0.5)
      
    else:
        print ("Data format error.")


def request_uri(uri,str_page):
    #header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0','Content-Type': 'application/x-www-form-urlencoded', 'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Encoding':'gzip, deflate','Referer':'http://search.appledaily.com.tw/appledaily/search'}
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0'}
    data = {"searchMode":"Adv","searchType":"text","querystrA":keyword,"select":"AND","source":"","sdate":start_date,"edate":end_date,"sorttype":"1","page":str_page}
    res = rs.post(uri, data=data, headers=header)
    html_data =  res.text
    return html_data

                        
def request_uri_content(uri):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0'}
    res = rs.get(uri, headers=header)
    html_data =  res.text
    return html_data


def parseAppleDailyNews(uri):
    # https://tw.appledaily.com/search/result?sort=time&searchType=all&dateStart=2018/07/01&dateEnd=2018/07/19&querystrS=川普&page=2
    html_data =  request_uri_content(uri)
    soup = bs(html_data,'html.parser')
    
    postdate = []
    link = []
    title = []
    body = []
    for div_soup in soup.findAll('div',attrs={"class":"content"}):
        for a_soup in div_soup.findAll('h2'):
            tmp_title = a_soup.text
            title.append(tmp_title.strip())
        for p_soup in div_soup.findAll('p'):
            body.append(p_soup.text)
        for li_soup in div_soup.findAll('li'):
            for a_soup in li_soup.findAll('a'):
                content_uri = a_soup.get('href').strip()
                link.append(content_uri)
                print(content_uri)
                html_data2 = request_uri_content(content_uri)
                soup2 = bs(html_data2,'html.parser')
                body_tmp = ''
                if len(soup2.findAll('div',attrs={"class":"ndArticle_margin"}))==0:
                    body.append('N/A')
                else:
                    for p_soup2 in soup2.findAll('div',attrs={"class":"ndArticle_margin"}):
                        body_tmp = p_soup2.getText().split('googletag')[0].replace('有話要說 投稿「即時論壇」','').split('.ndgKeyword')[0].split('，版權所有')[0].split('本新聞文字')[0].strip()
                        if 'confirmOMOAdvFlag' in body_tmp:
                            body_tmp = body_tmp.split('if')[0]
                        print(body_tmp)
                        body.append(body_tmp)
                if len(body_tmp)==0:
                    try:
                        tmp = (html_data2.split('Fusion.globalContent=')[1].split('promo_items')[0]+"}").split(',"}')[0]+"}"
                        body_tmp = ""
                        for k, v in enumerate(json.loads(tmp)["content_elements"]):
                            if k==0:
                                if 'content' in v:
                                    body_tmp = v['content'] 
                            elif k==1:
                                if 'content' in v:
                                    body_tmp = v['content'] 
                        print(body_tmp)
                        body.append(body_tmp)
                    except Exception as e:
                        pass
                #time.sleep(0.5)

    for time_soup in soup.findAll('time'):
        tmp_d = ''
        if len(time_soup.getText())==8:
            tmp_d = time_soup.getText()[0:4]+'-'+time_soup.getText()[4:6]+'-'+time_soup.getText()[6:8]
        #elif len(time_soup.getText())==5:
            #tmp_d = datetime.datetime.now().strftime('%Y-%m-%d')
        if len(tmp_d)>1:
            postdate.append(tmp_d)
    
    current = 0
    while current < len(postdate):
        items.append({
          "title": title[current],
          "link":link[current],
          "body":body[current],
          "postdate":postdate[current],
          #"updatetime":datetime.datetime.now(),  # MongoDB
          "updatetime":datetime.datetime.now().strftime('%Y-%m-%d')
        })
        current+=1
    #return items
    

if __name__ == '__main__':
    uri = 'https://tw.appledaily.com/search/result?sort=time&searchType=all&dateStart='+start_date.replace('-','%2F')+'&dateEnd='+end_date.replace('-','%2F')+'&querystrS='+quote(keyword)
    items = []
    start_requests(uri);
    row_json = json.dumps(items, ensure_ascii=False)
    file = codecs.open(urllib.parse.unquote(keyword)+'.json', 'w', encoding='utf-8')
    file.write(row_json)
    file.close()
    r = requests.get(url=uri, headers={'Connection':'close'})

    print("Done")


https://tw.appledaily.com/search/result?sort=time&searchType=all&dateStart=2020%2F04%2F14&dateEnd=2020%2F04%2F15&querystrS=%E8%82%BA%E7%82%8E, page=1
https://tw.appledaily.com/local/realtime/20200415/1731896/
刑事局中部打擊犯罪中心偵六隊在上月底接獲線報指出，台中市北屯區一名劉姓家庭主婦（30歲）透過臉書及Line等社團群組販售醫療用口罩，一片要價12到14元，趁亂大發國難財，警方立即啟動追查，發現劉女頻繁前往便利商店或郵局寄送口罩包裹，本月1日持搜索票逮人，並在其屋內搜出8681片醫療口罩。<br />&nbsp;<br />警方依據劉女供詞，循線查獲上游從事服務業的陳姓男子（39歲）、江姓男子（40歲）及負責送貨的白牌車陳姓司機（27歲），查扣798片口罩，進一步追查口罩來源，發現分別來自雲林一家被中央徵用的口罩工廠，查扣5萬810片醫療口罩，另在台中市一家專做外銷的口罩工廠，查扣醫療口罩1萬3720片，總計查扣7萬4009片口罩。<br />&nbsp;<br />中打偵六隊隊長王嘉華表示，4名嫌犯大發國難財賺外快，江男透過管道找上台中與雲林的口罩工廠的不肖員工，以每片5到7元購入，交由陳姓司機運送，以每片6到9元轉售陳男，陳男再以每片10元售予劉女，劉女則以12到14元價格對外販售，從上月中旬犯案2周，已售出超過3萬片口罩。<br />&nbsp;<br />王嘉華指出，訊後將4嫌依《傳染病防治法》、《嚴重特殊傳染性肺炎防治及紓困振興特別條例》及《藥事法》移送，經台中地檢署審理後裁定劉女15萬元、3名男嫌10萬元交保，另外也通知2家口罩工廠負責人到案，追查盜賣口罩員工身分，依法究辦。（王煌忠／台中報導）<br />&nbsp;<br />&nbsp;
https://tw.appledaily.com/finance/realtime/20200415/1731650/
 美國總統川普周一在記者會上表示，武漢肺炎新感染人數成長速度已趨穩，「明確證據」顯示舒緩措施已在美國奏效。紐約州長古莫（Andrew Cuomo）也對紐約州疫情發表樂觀看法，認為「最糟情

全球研發中的武漢肺炎疫苗多達70種，但目前尚無藥可醫，各國對相關藥物也正積極開發研究，近期由中興大學基因體暨生物資訊所教授侯明宏帶領的團隊，與喬治梅森大學找到針對冠狀病毒「核殼蛋白」的藥物策略，且證實它與現今開發中的幾種潛力藥物相比，具有「副作用低、廣效、跨物種」等優勢。<br />&nbsp;<br />侯明宏解釋，冠狀病毒屬於核糖核酸(RNA)病毒，能快速基因突變來逃脫藥物的毒殺，而病毒的遺傳物質RNA，主要透過「核殼蛋白」來保護，讓病毒利於複製、繁殖，「核殼蛋白」的突變機率低，這次開發出來的藥物5-benzyloxygramine（P3），能讓核殼蛋白失去作用，抑制病毒的複製。<br />&nbsp;<br />該藥物除了以武漢肺炎測試有效，與興大獸醫系合作測試鼠肝炎病毒（MHV）、豬流行性下痢病毒（PEDV）也同樣有效。侯明宏指出，很多冠狀病毒能夠跨物種的傳染，因此跨物種抑制傳播，也是很好的策略方向。<br /><br />現階段各國開發的幾個具有潛力的抗武漢肺炎藥物，有能抑制病毒複製病毒遺傳物酵素的「瑞德西韋」、能抑制病毒產生個別病毒蛋白酵素的「洛匹那韋」、干擾病毒感染且能調節免疫力的「奎寧」，而興大開發出來的藥物「P3」，為第一個針對「核殼蛋白」的高潛力藥物。<br /><br />經分析，藥物P3的治療指數是25.1，「治療指數只要大於10，就是具有開發潛力藥物的指標。」興大近期也與國內生技藥公司「華安醫學」簽署合作開發備忘錄，協同藥物開發。<br /><br />華安醫學董事長邱壬乙表示，每個新藥開發都要經過細胞試驗、動物試驗、人體試驗、臨床1至3期，開發成本粗估100億元以上，耗時數年，是非常漫長的過程，「對此藥物我們會優先開發，期望能將時程再縮短。」（洪子恩／台中報導）<br /><br />更多健康醫療新聞，詳見「<a href="https://bit.ly/2PB75bH">健康蘋道</a> 」<br />&nbsp;
https://tw.appledaily.com/finance/realtime/20200415/1731774/
 據美國財政部周二聲明，這份紓困協議涵蓋所有主要航空公司，政府將於先前公布的2.2兆美元（約66.16兆元台幣）經濟刺激方案（CARES Act）中，撥出250億美元（約7517.75億元台幣）用於紓困客

 <span style="color: rgb(0, 0, 255);"><strong> 本條目不斷更新</strong> </span> <br> 【下午3點35分】<br> 台股今日上漲百點，三大法人合計賣超台股28.77億元，自營商賣超（含避險）18.58億元，投信買超26.65億元，外資及陸資（含外資自營商）賣超36.84億元。<br> <br> 【中午1點32分】<br> 台積電上漲0.88%，加上其他中小型電子股同步反攻，可成上漲超過5%、微星漲逾3%、華新科更拉出漲停，另外，這波武肺疫情受創最深的觀光、旅行族群同步反撲，晶華及雄獅強攻漲停，終場指數上漲114點。<br> <br> 【上午11點11分】<br> 台新投顧副總黃文清表示，受惠美股大漲，台股開高走高，盤中10449點，近期有望突破10500點大關，主因市場預期歐美國家肺炎疫情漸趨緩，確診數達到高峰，緊接著民間消費會開始復甦活絡，加上美國總統川普喊出5月有望復工、接連提出振興方案。<br>  <br> 另一方面，台股法說陸續登場，也為台股注入活水，刺激投資人信心，今日預估成交量約1400~1500億。<br>  <br> 黃文清表示，盤面以電子股、5G相關概念、上游半導體有亮眼漲勢，此外，食品、防疫股短期也有機可循，金融股則是輪動補漲。他提醒，明日台積電法說會登場，備受市場關注，以及4月下旬至5月初美股財報周，影響短線盤面的表現。<br>  <br> 他進一步提到，雖然大廠上半年受疫情衝擊，營收不如預期，但整體來看今年還是會成長，惟需求面力道不明顯，但相信一旦疫情趨緩，投資信心堅定就會穩定，台股國安基金亦伺機而動，讓投資人情緒好轉、加強投資信心。<br>  <br> 不過，現階段大盤短線漲幅已超過20%，近期將有短線獲利回吐的賣壓，短線上還是有機會，但空間有限，或呈緩漲或區間震盪整理可能，投資操作策略，仍建議5G概念股，若是已出現跌深反彈，亦可適度獲利了結。<br> <br> 【上午9點55分】<br> 除了電子權值股之外，中小型電子股也同步走揚，國巨開盤雖因GDR折價發行股價重挫，但旋即急拉翻紅大漲逾6%，帶動台股上漲逾百點。<br> <br> 【上午9點1分】<br> 在台積電、大立光及鴻海電子股三王領軍下，電子股持續反攻，帶動台股開盤不久即上漲逾80點，突破10400點關卡。<br>